In [4]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
#all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

In [6]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embedding_model,
    persist_directory="./vectordb"
)

vectorstore.persist()  

C:\Users\Playdata\AppData\Local\Temp\ipykernel_14580\3089082706.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [7]:
vectorstore.get()

{'ids': ['b76214db-e973-478b-a689-fbf68cd35bd4',
  'fadb557d-982e-481e-b6b0-45c76708fd1e',
  '460ead8e-c6ea-4b7a-b119-debb9e96df15',
  '5804e7dd-3824-44b6-8e5f-09e78a47d85f',
  '971dff12-3dcb-4a86-891f-fce6a7d2569a',
  '5462fd23-3a5d-4518-b2e0-3c21af2ae079',
  '28f319f7-8e4b-4496-a25a-8502b6405f95',
  'dbbf757d-324b-48e7-bb6f-15ba92002988',
  '2d7c789a-5a0e-4652-8d3a-00c0a16cfb7a',
  '4e23b016-0352-4b0b-82ab-7aff4eadff56',
  '5d64d37f-b678-4b5e-bd56-717e2caffbba',
  'e39f6e6e-bf90-4df1-a6ba-5285b389bcfd',
  'dd0a3cc7-e6e9-424f-beeb-aa7afab44ffd',
  '8e63c3e3-5b3e-459b-828e-0bf52be86f42',
  '203f031f-cca0-4003-826c-20fc98e5aee3',
  '15730475-0eeb-4915-b148-951f98a76e4f',
  '323348c5-f942-4016-9ba2-4ba42e590100',
  'bf1d07ab-5679-4e64-8bd7-b80e09ef229b',
  '4096e015-86a2-4793-b501-44ef6ce6d8c3',
  'c65b4772-800c-4fff-9cde-e9633681aef5',
  'bafa57a6-796d-475f-a440-86c61b294209',
  '95d384c1-f3f3-48ff-8306-d4d139c9b21c',
  '065e456a-10f8-416d-b2d9-d2b3fb0cf6c0',
  '7259661b-1b3d-4670-84dc-

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [9]:
# 5. 실제 질의 실행
query = "86다카1551"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])

✅ 답변:
 - 용어/조항 정의: 대법원 86다카1551 판례는 차량이 진로를 변경하려고 할 때, 변경하려는 방향에서 오는 모든 뒤차와의 충돌을 피할 수 있는 충분한 거리를 확보하지 못하면 진로를 변경해서는 안 된다는 규정을 설명하고 있습니다. 따라서, 진로를 변경하려는 차량은 이러한 안전거리를 확보할 의무가 있습니다.
- 출처가 명시된 경우: 대법원 86다카1551 판례
